# Setup
- packages installation  
- imports

In [137]:
%pip install numpy pandas matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.


In [138]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [139]:
import cv2
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import os
import random

# Image Processing

In [140]:
DATASET_CARS_BR_PATH = "./dataset_RodoSol-ALPR/images/cars-br" # Path to images of cars with old license plate model
DATASET_CARS_ME_PATH = "./dataset_RodoSol-ALPR/images/cars-me" # Path to images of cars with MERCOSUL license plate model
DATASET_MOTORCYCLES_BR_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-br" # Path to images of motorcycles with old license plate model
DATASET_MOTORCYCLES_ME_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-me" # Path to images of motorcycles with MERCOSUL license plate model

VERBOSE = True

In [141]:
def verbose_decorator(func):
    def wrapper(*args, **kwargs):
        if VERBOSE:
            print()
            print(func)
        return func(*args, **kwargs)
    return wrapper

In [142]:
@verbose_decorator
def sample_images(data_path: str, exclude_path: str, sample_size=10) -> list[str]:
    """
    sample_images randomly chooses a number of images, equal to the parameter sample_size, from a directory specified by data_path 
    excluding images already at directory specified by exclude_path
    """
    excluded_names= [file_name for file_name in os.listdir(exclude_path) if file_name.endswith(".jpg")]

    image_names = [file_name for file_name in os.listdir(data_path) if file_name.endswith(".jpg") and file_name not in excluded_names]

    image_names = random.sample(image_names, min(sample_size, len(image_names)))

    if VERBOSE:
        print(f"Excluded files from sampling: {excluded_names}")
        print(f"Sampled files: {image_names}")
    
    return image_names

In [152]:
@verbose_decorator
def show_comparison(original_images, processed_images):
    """
    show_comparison shows a comparison side by side between original_images and processed_images
    """
    images = [(orig, proc) for orig, proc in zip(original_images, processed_images)]
    if VERBOSE:
        print(f"images zipped structure: {[(orig[1], proc[1]) for orig, proc in images]}")
        for orig, proc in images:
            print(f"{orig[1].split('/')[-1]} shape: {orig[0].shape}")
            print(f"{proc[1]} shape: {proc[0].shape}")
    
    fig, axes = plt.subplots(len(images), 2, figsize=(20, 10))
    ax = axes.ravel()
    
    ax_i = -1
    for orig, proc in images:
        ax_i += 1
        ax[ax_i].imshow(orig[0])
        ax[ax_i].set_title(orig[1].split('/')[-1])
        ax[ax_i].set_axis_off()
        
        ax_i += 1
        ax[ax_i].imshow(proc[0])
        ax[ax_i].set_title(proc[1].split('/')[-1])
        ax[ax_i].set_axis_off()


In [166]:
@verbose_decorator
def sample_execute_save_show(data_path: str, save_path: str, algorithm: callable, algorithm_args: dict, show=True, pre_sample=None, sample_size=10):
    """
    sample_execute_save_show randomly sample images from directory specified by data_path ignoring 
    images already at directory specified by save_path, execute a algorithm of image processing which must return a single image,
    save processed images at a directory specified by save_path and then shows a comparison between the original and processed image.
    If presample contains a valid images names list then execute algorithm with this images. 
    """
    os.makedirs(save_path, exist_ok=True) # Create directory and dont raise error if already exists
    is_pre_sample = isinstance(pre_sample, list) and all([sample in os.listdir(data_path) for sample in pre_sample]) # If is a list and directory specified by data_path contain ALL samples.

    if is_pre_sample: 
        images_name = pre_sample
    else:
        images_name = sample_images(data_path, save_path, sample_size)

    images_path = [data_path + "/" + image_name for image_name in images_name]

    original_images = [(cv2.imread(image_path), image_path) for image_path in images_path]

    processed_images = []
    for original_image, path in original_images:
        image_save_path = save_path + '/' + path.split('/')[-1]

        processed_image = algorithm(original_image, **algorithm_args)
        processed_images.append( (processed_image, image_save_path) )

        
        cv2.imwrite(image_save_path, processed_image)
        if VERBOSE:
            print(f"{path.split('/')[-1]} saved at {image_save_path}")

    if show:
        show_comparison(original_images, processed_images)

    if VERBOSE:
        print(f"is pre sampled: {is_pre_sample}")
    return processed_images

## Downscaling

In [157]:
SAVE_CARS_BR_DOWNSCALING_PATH = "./output/images/downscaling/cars-br" # Save path to downscaled images of cars with old license plate model
SAVE_CARS_ME_DOWNSCALING_PATH = "./output/images/downscaling/cars-me" # Save path to downscaled images of cars with MERCOSUL license plate model
SAVE_MOTORCYCLES_BR_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-br" # Save path to downscaled images of motorcycles with old license plate model
SAVE_MOTORCYCLES_ME_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-me" # Save path to downscaled images of motorcycles with MERCOSUL license plate model

DOWNSCALING_RATIO = 0.75

In [160]:
@verbose_decorator
def resize(image, ratio, name, flag):
    width = int(image.shape[1] * ratio)
    height = int(image.shape[0] * ratio)
    dim = (width, height)

    if VERBOSE:
        print(f"interpolation method: {name}")
        print(f"downscaling ratio: {str(ratio)}")
        print(f"original image shape: {image.shape}")
        print(f"new shape: {dim}")
        
    return cv2.resize(image, dim, interpolation=flag)

In [167]:
INTERPOLATION_FLAGS = [
    ('bilinear', cv2.INTER_LINEAR),
    ('bicubic', cv2.INTER_CUBIC),
    ('lanczos', cv2.INTER_LANCZOS4)
]

pre_sample = sample_images(DATASET_CARS_BR_PATH, SAVE_CARS_BR_DOWNSCALING_PATH + "/" + 'bilinear', 10)
for name, flag in INTERPOLATION_FLAGS:
    x = sample_execute_save_show(DATASET_CARS_BR_PATH, SAVE_CARS_BR_DOWNSCALING_PATH + "/" + name, 
                                algorithm=resize, 
                                algorithm_args={"ratio": DOWNSCALING_RATIO, "name": name, "flag": flag},
                                show=False,
                                pre_sample=pre_sample,
                                sample_size=3)


<function sample_images at 0x000002D25D89CF70>


FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: './output/images/downscaling/cars-br/bilinear'